# 1. TO UPDATE Setup

In [1]:
!pip install matplotlib
import matplotlib.pyplot as plt
import keras.optimizers as optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from keras import models
from keras import regularizers
import math

2023-01-05 13:17:44.452174: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# this jupyter notebook contains the code used to analyze the baseline approach

In [2]:
# image parameters
input_height = 69
input_width = 69

# augmentation parameters
rescale = True
if rescale:
    rescale_size=1./255
else:
    rescale_size=1
augmentation=False

rotation_range=40
width_shift_range=0.2
height_shift_range=0.1
shear_range=0.2
zoom_range=0.2
horizontal_flip=True
fill_mode='nearest'

In [3]:
# best loss function for multi-class classification, measures the distance between two probability distributions
# the probability distribution of the output of the network and the true distribution of the labels
loss_function='categorical_crossentropy'

optimizer='rmsprop'
optimizer_learning_rate=1e-4
epochs=100
batch_size=32
num_classes = 9
regularizer=regularizers.l1_l2(l1=0.001, l2=0.001) # simultaneous l1 and l2, add 0.001*weight_coefficient_value + 0.001 * 1/2*weight^2

if optimizer == 'rmsprop':
    optimizer=optimizers.RMSprop(learning_rate=optimizer_learning_rate)

In [ ]:
# customized metrics for multi class classification
import tensorflow as tf
import keras.backend as K
# best loss function for multi-class classification, measures the distance between two probability distributions
# the probability distribution of the output of the network and the true distribution of the labels
loss_function='categorical_crossentropy'

def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

metrics = [
    precision,
    recall,
    tf.keras.metrics.CategoricalAccuracy(name='acc')
]

# 2. Holdout method

In [4]:
from sklearn.model_selection import train_test_split
import pandas as pd
import shutil
import os

In [5]:
paths = {
    'TRAIN_PATH' : os.path.join('workspace', 'images', 'train'),
    'TEST_PATH' : os.path.join('workspace', 'images','test'),
    'EVAL_PATH' : os.path.join('workspace', 'images','eval'),
    'IMAGES_PATH': os.path.join('workspace','images','all'),
    'ANNOTATION_PATH': os.path.join('workspace','annotations'),
    'LOG_DIR' : os.path.join('model', 'log_dir')
 }

In [6]:
# create paths
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

In [7]:
# remove all files already present for an old holdout method execution
for i in range(0,10):
    for file in os.listdir(os.path.join(paths["TRAIN_PATH"],str(i))):
        if file != ".DS_Store":
            os.remove(os.path.join(paths["TRAIN_PATH"],str(i), file))
    for file in os.listdir(os.path.join(paths["TEST_PATH"],str(i))):
        if file != ".DS_Store":
            os.remove(os.path.join(paths["TEST_PATH"],str(i), file))
    for file in os.listdir(os.path.join(paths["EVAL_PATH"],str(i))):
        if file != ".DS_Store":
            os.remove(os.path.join(paths["EVAL_PATH"],str(i), file))

df = pd.read_csv(os.path.join(paths['ANNOTATION_PATH'],"annotations.csv"))
df = df.dropna()

for i in range(0,10):
    if not os.path.exists(os.path.join(paths["EVAL_PATH"],str(i))):
        if os.name == 'posix':
            !mkdir -p {os.path.join(paths["EVAL_PATH"],str(i))}
    if not os.path.exists(os.path.join(paths["TRAIN_PATH"],str(i))):
        if os.name == 'posix':
            !mkdir -p {os.path.join(paths["TRAIN_PATH"],str(i))}
    if not os.path.exists(os.path.join(paths["TEST_PATH"],str(i))):
        if os.name == 'posix':
            !mkdir -p {os.path.join(paths["TEST_PATH"],str(i))}

# split in training set (90%) and test set (10%)
X_train, X_test, y_train, y_test = train_test_split(df, df, test_size=0.1, shuffle=True)
print("Train size: " + str(len(X_train)))

for index, row in X_test.iterrows():
    shutil.copy(os.path.join(paths["IMAGES_PATH"],str(row['Path'])),
                os.path.join(paths["TEST_PATH"],str(row['Class']),str(row['Path'])))

# split the training set into the final training set (80%) and validation set (20%)
X_train, X_eval, y_train, y_eval = train_test_split(X_train, y_train, test_size=0.2, shuffle=True)
print("Test size: " + str(len(X_test)))
print("Eval size: " + str(len(X_eval)))

for index, row in X_train.iterrows():
    shutil.copy(os.path.join(paths["IMAGES_PATH"],str(row['Path'])),
                os.path.join(paths["TRAIN_PATH"],str(row['Class']),str(row['Path'])))

for _, row in X_eval.iterrows():
    shutil.copy(os.path.join(paths["IMAGES_PATH"],str(row['Path'])),
                os.path.join(paths["EVAL_PATH"],str(row['Class']),str(row['Path'])))

FileNotFoundError: [Errno 2] No such file or directory: 'workspace/images/train/9'

In [ ]:
# training set image data generator
if augmentation:
    train_datagen = ImageDataGenerator(
          rescale=rescale_size,
          rotation_range=rotation_range,
          width_shift_range=width_shift_range,
          height_shift_range=height_shift_range,
          shear_range=shear_range,
          zoom_range=zoom_range,
          horizontal_flip=horizontal_flip,
          fill_mode=fill_mode)
else:
    train_datagen = ImageDataGenerator(rescale=1./255)

train_dir=paths['TRAIN_PATH']
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(input_width, input_height), batch_size=batch_size, class_mode='categorical')

In [ ]:
# validation set image data generator
val_datagen = ImageDataGenerator(rescale=rescale_size) # it should not be augmented

validation_dir=paths['EVAL_PATH']
validation_generator = val_datagen.flow_from_directory(validation_dir, target_size=(input_width, input_height), batch_size=batch_size, class_mode='categorical')

# 3. Model creation

In [ ]:
# Function used to create the CNN structure used for the classification
def create_cnn(width, height, depth, num_classes,filters=(16, 32, 64)):
    inputShape = (height, width, depth)
    chanDim = -1
    model = models.Sequential()
    for (i, f) in enumerate(filters):
        # first CONV layer set appropriately
        if i == 0:
            model.add(Conv2D(f, (3, 3), activation="relu", padding="same", kernel_regularizer=regularizer, input_shape=inputShape))
        else:
            model.add(Conv2D(f, (3, 3), activation="relu", padding="same", kernel_regularizer=regularizer))
        # with padding = same we are able to center convolutional windows around every input tile, in order to have always the same size of the input image
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
    # FLATTEN => FC => RELU => BN => DROPOUT
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(64, activation="relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Dense(num_classes, activation="softmax"))
    return model

model = create_cnn(input_width, input_height, 3, 10, (8,16))
model.summary()

In [ ]:
# compile the model
model.compile(optimizer=optimizer,
              loss=loss_function,
              metrics=metrics)

# 4. Model training

In [ ]:
# steps_per_epoch: number of batches to be drawn from the generator after assuming epoch over
# epochs: number of epochs
# validation_steps: how many batches to draw from the validation generator for evaluation

number_training = len(X_train)
number_eval = len(X_eval)

history = model.fit(
      train_generator,
      steps_per_epoch=int(math.ceil((1. * number_training) / batch_size)),
      epochs=epochs,
      validation_data=validation_generator,
      validation_steps=int(math.ceil((1. * number_eval) / batch_size)))

# 5. Visualization

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

In [ ]:
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation ACC')
plt.legend()
plt.figure()

In [ ]:
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
precision = history.history['precision']
val_precision = history.history['val_precision']
plt.plot(epochs, precision, 'r', label='Training precision')
plt.plot(epochs, val_precision, 'b', label='Validation precision')
plt.title('Training and validation Precision')
plt.legend()
plt.figure()

In [ ]:
recall = history.history['recall']
val_recall = history.history['val_recall']
plt.plot(epochs, recall, 'r', label='Training recall')
plt.plot(epochs, val_recall, 'b', label='Validation recall')
plt.title('Training and validation Recall')
plt.legend()
plt.figure()

In [ ]:
# smooth curves if they look noisy
# replace each point with an exponential moving average of the previous points
def smooth_curve(points, factor=0.8):
  smoothed_points = []
  for point in points:
    if smoothed_points:
      previous = smoothed_points[-1]
      smoothed_points.append(previous * factor + point * (1 - factor))
    else:
      smoothed_points.append(point)
  return smoothed_points

In [ ]:
plt.plot(epochs,
         smooth_curve(acc), 'r', label='Smoothed training acc')
plt.plot(epochs,
         smooth_curve(val_acc), 'b', label='Smoothed validation acc')
plt.title('Training and validation MAE')
plt.legend()
plt.figure()

In [ ]:
plt.plot(epochs,
         smooth_curve(loss), 'r', label='Smoothed training loss')
plt.plot(epochs,
         smooth_curve(val_loss), 'b', label='Smoothed validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()
# display average, the model may improve even if not reflected

# 6. Early stopping

In [ ]:
import keras
import os

callbacks_list = [
        # interrupts training when accuracy has stopped improving accuracy on the validation set for at least 3+1=4 epochs
        keras.callbacks.EarlyStopping(
            monitor='acc', # should be part of the metrics specific during compilation
            patience=3,
        ),
        # save the current weights after every epoch
        #keras.callbacks.ModelCheckpoint(
        #    filepath=os.path.join(paths['MODELS'],'CNN_baseline.h5'),
        #    monitor='val_loss', # do not overwrite until val_loss is improved
        #    save_best_only=True
        #),
        # monitor the model's validation loss and reduce the LR when the validation loss has stopped improving, effective strategy to escape local minima
        #keras.callbacks.ReduceLROnPlateau(
        #    monitor='val_loss',
        #    factor=0.2, # divides LR by 5 when triggered
        #    patience=3 # called when stopped improving for 3 epochs
        #),
        #keras.callbacks.TensorBoard(
        #    log_dir=paths['LOG_DIR'],
        #    write_graph=True,
        #    histogram_freq=1 # record activation histograms every 1 epoch
        #)
]

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=int(math.ceil((1. * number_training) / batch_size)),
      epochs=200,
      validation_data=validation_generator,
      callbacks=callbacks_list,
      validation_steps=int(math.ceil((1. * number_eval) / batch_size)))

# 7. Model testing

In [ ]:
number_test = 0
for i in range(9):
      path = os.path.join(paths['TEST_PATH'],str(i))
      n_images = len([f for f in os.listdir(path)if os.path.isfile(os.path.join(path, f))])
      number_test += n_images

In [ ]:
# test set image generator
test_dir=paths['TEST_PATH']
test_datagen = ImageDataGenerator(rescale=rescale_size) # it should not be augmented

test_generator = test_datagen.flow_from_directory(test_dir, target_size=(input_width, input_height), batch_size=batch_size, class_mode='categorical', classes=None, shuffle=False)

In [ ]:
# confusion matrix and classification report
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

test_generator.reset()
Y_pred = model.predict_generator(test_generator, number_test // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['0','1','2','3','4','5','6','7','8']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

In [ ]:
# ROC curve and AUC
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, roc_auc_score, auc
fig, c_ax = plt.subplots(1,1, figsize = (12, 8))

def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)

    for (idx, c_label) in enumerate(target_names):
        fpr, tpr, thresholds = roc_curve(y_test[:,idx].astype(int), y_pred[:,idx])
        c_ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
    c_ax.plot(fpr, fpr, 'b-', label = 'Random Guessing')
    c_ax.legend()
    return roc_auc_score(y_test, y_pred, average=average)

test_generator.reset()
y_pred = model.predict_generator(test_generator, verbose = True)
y_pred = np.argmax(y_pred, axis=1)
print("Multiclass roc auc score:", multiclass_roc_auc_score(test_generator.classes, y_pred))

# 8. Model exportation

In [ ]:
model.save("models/CNN_baseline.h5")

# 9. Plot model as graph of layers

In [ ]:
from keras.utils import plot_model

In [ ]:
plot_model(model, show_shapes=True, to_file='model.png')